# TEST

Author: Carlos Gómez de Olea, Professorship of Lunar and Planetary Exploration @TUM, 2025

In [ ]:
using BenchmarkTools
using JET
using Distributions

include("../src/ExosPhID.jl")

photodestruction (generic function with 1 method)

## 1. Test photodestruction function for a normal use case considering different species

In [2]:
reaction_occurence, reaction_name, product_types, product_velocities = true, "", [], []

(true, "", Any[], Any[])

In [3]:
dt = 1e10 # s, Time stepo necessary for a great number of simulations
solar_activity = 0.0
velocities = Dict("H2O" => 600.0, "OH" => 650.0, "H2" => 1900.0, "H" => 2700.0, "H(-)"=> 2700.0, "HO2" => 500.0, "H2O2" => 500.0, "He" => 1760.0, "Ne" => 560.0)


# for parent_name in ["H2O", "OH", "H2", "H", "H(-)", "HO2", "H2O2", "He", "Ne"]
for parent_name in ["H2O", "OH", "H2"]


    print("\nStarting test for $(parent_name)\n")

    if parent_name == "H2O"
        quiet_rate = 13.23e-6 # Calculated by Carlos Gomez de Olea (2025)
        active_rate = 22.21e-6 # Calculated by Carlos Gomez de Olea (2025)
    elseif parent_name == "OH"
        quiet_rate = 22.45e-6 # Calculated by Carlos Gomez de Olea (2025)
        active_rate = 32.56e-6 # Calculated by Carlos Gomez de Olea (2025)
    elseif parent_name == "H2"
        quiet_rate = 14.24e-6 # Calculated by Carlos Gomez de Olea (2025)
        active_rate = 32.83e-6 # Calculated by Carlos Gomez de Olea (2025)
    elseif parent_name == "H"
        quiet_rate = 7e-8 # Carlos Gomez de Olea (2026)
        active_rate = 1.68e-7 # Carlos Gomez de Olea (2026)
    elseif parent_name == "H(-)"
        quiet_rate = 14.24 # Carlos Gomez de Olea (2026)
        active_rate = 14.24 # Carlos Gomez de Olea (2026)
    elseif parent_name == "HO2"
        quiet_rate = 6.59e-3    # Gomez de Olea (2026)
        active_rate = 6.59e-3
    elseif parent_name == "H2O2"
        quiet_rate = 1.43e-4    # Gomez de Olea (2026)
        active_rate = 1.43e-4
    elseif parent_name == "He"
        quiet_rate = 5.57e-8    # Gomez de Olea (2026)
        active_rate = 16.8e-8
    elseif parent_name == "Ne"
        quiet_rate = 1.72e-7    # Gomez de Olea (2026)
        active_rate = 3.75e-8
    end

    k = (1-solar_activity)*quiet_rate + solar_activity*active_rate # Photodestruction rate

    num_reactions = Int(round(k*dt))

    parent_vector = fill(parent_name, num_reactions)

    progress_interval = num_reactions ÷ 10  # Every 10%
    next_progress = progress_interval       # When to print next update

    for (index, parent_type) in enumerate(parent_vector)

        if parent_type in ("H2O", "H2", "OH", "H", "H(-)", "HO2", "H2O2", "He", "Ne")
            θ_parent, φ_parent = 2 * π * rand(), π * rand()
            parent_velocity =  velocities[parent_type] .*  (sin(φ_parent) * cos(θ_parent), sin(φ_parent) * sin(θ_parent), cos(φ_parent))
        else
            print("Parent species is not valid")
        end

        reaction_occurence, reaction_name, product_types, product_velocities, wvl_range = photodestruction(solar_activity, dt, parent_type, parent_velocity, nothing)
        
        if index == next_progress || index == num_reactions
            println("$(parent_name) Progress: $(round(100 * index / num_reactions; digits=1))% reactions processed.")
            next_progress += progress_interval
        end
    end

    print("\nTest completed successfully for $(parent_name)\n")
end


Starting test for H2O
H2O Progress: 10.0% reactions processed.
H2O Progress: 20.0% reactions processed.
H2O Progress: 30.0% reactions processed.
H2O Progress: 40.0% reactions processed.
H2O Progress: 50.0% reactions processed.
H2O Progress: 60.0% reactions processed.
H2O Progress: 70.0% reactions processed.
H2O Progress: 80.0% reactions processed.
H2O Progress: 90.0% reactions processed.
H2O Progress: 100.0% reactions processed.

Test completed successfully for H2O

Starting test for OH
OH Progress: 10.0% reactions processed.
OH Progress: 20.0% reactions processed.
OH Progress: 30.0% reactions processed.
OH Progress: 40.0% reactions processed.
OH Progress: 50.0% reactions processed.
OH Progress: 60.0% reactions processed.
OH Progress: 70.0% reactions processed.
OH Progress: 80.0% reactions processed.
OH Progress: 90.0% reactions processed.
OH Progress: 100.0% reactions processed.

Test completed successfully for OH

Starting test for H2
H2 Progress: 10.0% reactions processed.
H2 Progr

## 2. Numerical Benchmarking with Benchmark Tools

In [4]:
# for parent_name in ["H2O", "OH", "H2", "H", "H(-)", "HO2", "H2O2", "He", "Ne"]
for parent_name in ["H2O"]

    print("\nStarting Numerical Benchmark for $(parent_name)\n")

    parent_vector = fill(parent_name, 1)

    for (index, parent_type) in enumerate(parent_vector)

        if parent_type in ("H2O", "H2", "OH", "H", "H(-)", "HO2", "H2O2", "He", "Ne")
            θ_parent, φ_parent = 2 * π * rand(), π * rand()
            parent_velocity =  velocities[parent_type] .*  (sin(φ_parent) * cos(θ_parent), sin(φ_parent) * sin(θ_parent), cos(φ_parent))
        else
            print("Parent species is not valid")
        end

        bm = @benchmark photodestruction($solar_activity, $dt, $parent_type, $parent_velocity, nothing)
        display(bm)
    end

    print("\nNumerical Benchmark completed for $(parent_name)\n")

end

BenchmarkTools.Trial: 10000 samples with 1 evaluation per sample.
 Range (min … max):  11.688 μs …  37.056 ms  ┊ GC (min … max):  0.00% … 99.62%
 Time  (median):     17.794 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   26.250 μs ± 370.560 μs  ┊ GC (mean ± σ):  14.06% ±  1.00%

    ▁██▆▄▄▂▅▅▆▅▄▃▂▁▁                                           ▂
  ▃▄██████████████████████▇▇▇▆▇▆▆▆▆▆▆▆▅▅▆▆▅▆▅▆▆▅▄▅▅▆▅▅▅▄▄▄▃▅▄▄ █
  11.7 μs       Histogram: log(frequency) by time      82.3 μs <

 Memory estimate: 15.46 KiB, allocs estimate: 44.


Starting Numerical Benchmark for H2O

Numerical Benchmark completed for H2O


## 3. Detect Potential bugs and type instabilities with JET

### 3.1 Detect type instability with @report_opt

In [5]:
parent_name = "H2O"
parent_vector = fill(parent_name, 1)

for (index, parent_type) in enumerate(parent_vector)

    if parent_type in ("H2O", "H2", "OH", "H", "H(-)", "HO2", "H2O2", "He", "Ne")
        θ_parent, φ_parent = 2 * π * rand(), π * rand()
        parent_velocity =  velocities[parent_type] .*  (sin(φ_parent) * cos(θ_parent), sin(φ_parent) * sin(θ_parent), cos(φ_parent))
    else
        print("Parent species is not valid")
    end

    @report_opt photodestruction(solar_activity, dt, parent_type, parent_velocity, nothing)    
end

### 3.2 Detect type errors with @report_call

In [6]:
parent_name = "H2O"
parent_vector = fill(parent_name, 1)

for (index, parent_type) in enumerate(parent_vector)

    if parent_type in ("H2O", "H2", "OH", "H", "H(-)", "HO2", "H2O2", "He", "Ne")
        θ_parent, φ_parent = 2 * π * rand(), π * rand()
        parent_velocity =  velocities[parent_type] .*  (sin(φ_parent) * cos(θ_parent), sin(φ_parent) * sin(θ_parent), cos(φ_parent))
    else
        print("Parent species is not valid")
    end

    @report_call photodestruction(solar_activity, dt, parent_type, parent_velocity, nothing)    
end

### 3.3 Complete analsysis with @code_warntype

In [7]:
parent_name = "H2O"
parent_vector = fill(parent_name, 1)

for (index, parent_type) in enumerate(parent_vector)

    if parent_type in ("H2O", "H2", "OH", "H", "H(-)", "HO2", "H2O2", "He", "Ne")
        θ_parent, φ_parent = 2 * π * rand(), π * rand()
        parent_velocity =  velocities[parent_type] .*  (sin(φ_parent) * cos(θ_parent), sin(φ_parent) * sin(θ_parent), cos(φ_parent))
    else
        print("Parent species is not valid")
    end
  
    @code_warntype photodestruction(solar_activity, dt, parent_type, parent_velocity, nothing)
end

MethodInstance for photodestruction(::Float64, ::Float64, ::String, ::Tuple{Float64, Float64, Float64}, ::Nothing)
  from photodestruction(solar_activity::Float64, dt::Union{Float64, Int64}, parent_type::String, parent_velocity::Union{Float64, Tuple{Float64, Float64, Float64}}, sun_tuple::Union{Nothing, Tuple{Float64, Float64, Float64}}) @ Main ~/Desktop/TUM_MASTERS/11-MASTER THESIS/3 Code/photodestruction_main.jl:64
Arguments
  #self#::Core.Const(photodestruction)
  solar_activity::Float64
  dt::Float64
  parent_type::String
  parent_velocity::Tuple{Float64, Float64, Float64}
  sun_tuple::Core.Const(nothing)
Locals
  @_7::Union{Nothing, Tuple{Tuple{Int64, Tuple}, Tuple{Int64, Int64}}}
  #12::var"#12#15"{String}
  @_9::Int64
  reaction_occurence::Bool
  wavelength_range::Tuple{Vararg{Tuple}}
  reaction_types::Tuple{Vararg{String}}
  reaction_names::Tuple{Vararg{String}}
  reaction_probabilities::Tuple
  species_names::Tuple{Vararg{Tuple}}
  tsh_energies::Union{Float64, Tuple}
  type_in